### In this notebook, we'are going to cover some of the most fundamental concepts of tensors using TensorFlow

#### More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercise to try for yourself

### Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.16.2


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

2025-03-24 12:21:16.743596: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-03-24 12:21:16.743770: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-24 12:21:16.743780: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-24 12:21:16.743995: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-24 12:21:16.744011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of a tensor (`ndim` stands for number of dimensions)
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [6]:
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [7]:
# Check out the dimension of our vector
vector.ndim

1

In [8]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [9]:
matrix.ndim

2

In [10]:
# Create another matrix
another_matrix = tf.constant([[1., 2., 3.],
    [4., 5., 6.],
    [7., 8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 3), dtype=float16, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float16)>

In [11]:
# What's the ndim of the another_matrix
another_matrix.ndim

2

In [12]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
    [4, 5, 6]],
    [[7, 8, 9],
    [10, 11, 12]],
    [[13, 14, 15],
    [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [13]:
tensor.ndim

3

### What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

---
### Creating tensors with `tf.Variable`

In [14]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [15]:
# Create the same tensor with tf.Variable() as above

changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor


(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [16]:
# Let's try change one of the elements in our changeable tensor 

changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

This error happens because changeable_tensor is likely a TensorFlow tf.Variable, which does not support item assignment using standard Python syntax like changeable_tensor[0] = 7.

✅ Aaron's Notes - Why it happens:
tf.Variable is a mutable container for TensorFlow tensors, but you can't modify it in-place with normal indexing. Instead, TensorFlow provides specific methods to change its contents, such as `assign`, `assign_sub`, `assign_add`

In [17]:
# How about we try .assign()

changeable_tensor[0].assign(99)
changeable_tensor


<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([99,  7], dtype=int32)>

In [18]:
# Let's try change our unchangeable tensor

unchangeable_tensor[0] = 7
unchangeable_tensor

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [19]:
# Let's try `assign` to unchangeable constant

unchangeable_tensor[0].assign(99)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

#### 🔑 Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

#### Random tensors are tensors of some abitrary size which contain random numbers

In [23]:
# Create two random (but the same) tensors

random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.75658023, -0.06854693],
        [ 0.07595028, -1.2573844 ],
        [-0.23193759, -1.8107857 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.75658023, -0.06854693],
        [ 0.07595028, -1.2573844 ],
        [-0.23193759, -1.8107857 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [26]:
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

### Shuffle the order of elements in a tensor

In [33]:
# Shuffle a tensor (valueable for when you want to shuffle your data so the inherent order doesn't effect learning)

not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])
                        
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [38]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [39]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### 🛠️ **Exercise:** 
1. Read through TensorFlow documentation on [random seed generation](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)
2. And, practice writing 5 random tensors and shufffle them.

In [ ]:
# Ex1: random tensors and shuffled (this is eager mode)
tf.random.set_seed(42)

print("before shuffling:", 30*"-")
r1 = tf.random.uniform([3, 2])
r2 = tf.random.uniform([3, 2])
print(r1, r2)

print("\nafter shuffling:", 30*"-")
print(tf.random.shuffle(r1))
print(tf.random.shuffle(r2))

before shuffling: ------------------------------
tf.Tensor(
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]
 [0.03366041 0.68467236]], shape=(3, 2), dtype=float32) tf.Tensor(
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]
 [0.73115396 0.89256823]], shape=(3, 2), dtype=float32)

after shuffling: ------------------------------
tf.Tensor(
[[0.03366041 0.68467236]
 [0.3528825  0.46448255]
 [0.6645621  0.44100678]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.73115396 0.89256823]
 [0.9309944  0.252187  ]
 [0.68789124 0.48447883]], shape=(3, 2), dtype=float32)


In [ ]:
# Ex2 - shuffle with decorators! (This is graph mode)

tf.random.set_seed(1234)

@tf.function
def shuffle_test():
    print("before shuffling:", 30*"-")
    r1 = tf.random.uniform([3, 2])
    r2 = tf.random.uniform([3, 2])
    print(r1, r2)
    print("\nafter shuffling:", 30*"-")
    print(tf.random.shuffle(r1))
    print(tf.random.shuffle(r2))

shuffle_test()

before shuffling: ------------------------------
Tensor("random_uniform/RandomUniform:0", shape=(3, 2), dtype=float32) Tensor("random_uniform_1/RandomUniform:0", shape=(3, 2), dtype=float32)

after shuffling: ------------------------------
Tensor("RandomShuffle:0", shape=(3, 2), dtype=float32)
Tensor("RandomShuffle_1:0", shape=(3, 2), dtype=float32)


2025-03-24 15:12:14.985377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [56]:
# Ex3 - Eager Mode Example:

import tensorflow as tf

# Set seed for reproducibility
tf.random.set_seed(42)

# Create 5 random tensors and put them into a list
tensors = [tf.random.uniform([2, 2]) for _ in range(3)]

print("Before shuffling:", 20*"=")
for i, t in enumerate(tensors):
    print(f"Tensor {i}: \n{t} \n")
    
# Stack them into a single tensor so we can shuffle them
stacked = tf.stack(tensors)  # shape = (5, 2, 2)
shuffled = tf.random.shuffle(stacked)

print("After shuffling:", 20*"=")
for i, t in enumerate(shuffled):
    print(f"Tensor {i}: \n{t} \n")

Before shuffling: ====================
Tensor 0: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

After shuffling: ====================
Tensor 0: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 



### The above exercise, the **shuffle only happens across the first dimension,**, i.e., **between Tensor 0, 1, and 2**  -- not insdie each individual 2x2 tensor.

Let's say we have:
```python
tensors = [A, B, C]  # where each is a 2×2 tensor
stacked = tf.stack(tensors)  # shape = [3, 2, 2]
```

This give us a 3D tensor like:
```lua
[
  [[a1, a2],
   [a3, a4]],   # Tensor 0

  [[b1, b2],
   [b3, b4]],   # Tensor 1

  [[c1, c2],
   [c3, c4]]    # Tensor 2
]
```
When we call:
`tf.random.shuffle(stacked)`

We only are shuffling **the outer dimension** (axis 0), i.e., we're just changing the order of Tensor 0, Tensor 1, Tensor 2. as below:

```lua
→
[
  [[c1, c2],
   [c3, c4]],   # Was Tensor 2

  [[a1, a2],
   [a3, a4]],   # Was Tensor 0

  [[b1, b2],
   [b3, b4]]    # Was Tensor 1
]
```

### ✅ **What if we want to shuffle inside** each tensor?

In [59]:
# shuffle inside

print("Before shuffling INSIDE TENSOR:", 20*"=")
for i, t in enumerate(tensors):
    print(f"Tensor {i}: \n{t} \n")

shuffled_inside = [tf.random.shuffle(t) for t in tensors]

print("After shuffling INSDIE TENSOR:", 20*"=")
for i, t in enumerate(shuffled_inside):
    print(f"Tensor {i}: \n{t} \n")

Before shuffling INSIDE TENSOR: ====================
Tensor 0: 
[[0.6645621  0.44100678]
 [0.3528825  0.46448255]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.7413678  0.62854624]
 [0.01738465 0.3431449 ]] 

After shuffling INSDIE TENSOR: ====================
Tensor 0: 
[[0.3528825  0.46448255]
 [0.6645621  0.44100678]] 

Tensor 1: 
[[0.68789124 0.48447883]
 [0.9309944  0.252187  ]] 

Tensor 2: 
[[0.01738465 0.3431449 ]
 [0.7413678  0.62854624]] 



In [60]:
tf.random.shuffle(not_shuffled, seed=42)


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [61]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

### if set global seed
#### If both the global and the operation seed are set: 
> - Both seeds are used in conjunction to determine the random sequence

### The above setting - **Both global and op-level seed are set** has the following behavior:

> #### - this is the `most stable` and `deterministi` setup
> #### - Combines both seeds deterministic to generate the result
> #### - We'll get `the same output` every time, even accross many random ops, and more robust to TensorFlow version changes!

In [63]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [64]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [65]:
tf.random.set_seed(42)  # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### ⬆️ same result each time

#### It looks like if we want our shuffled tensors to be in the same order, we've got6 to use the global level random seed as the operation level random seed:

> **Rule 4**: if both the global and the op-seed are set: Both seeds are used in conjunmtion to determine the random sequence.

### Other ways to make tensors

In [67]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [68]:
# Create a tensor of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

#### The mian difference between Numpy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [70]:
# We can also turn NumPy arrays into tensors

import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)  # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # cpaital for matrix or tensor
# y = tf.constant(vector)  # non-cap0ital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [71]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

### 🔍 So What's the Difference?

| Feature                     | `tf.constant()`                 | `tf.convert_to_tensor()`        |
|----------------------------|----------------------------------|----------------------------------|
| Creates a constant tensor  | ✅ Yes                           | ✅ Yes                            |
| Accepts NumPy, list, etc.  | ✅ Yes                           | ✅ Yes                            |
| Optimized for constants    | ✅                                | ❌ (more general)                |
| Used internally in APIs    | ❌                                | ✅                                |
| Can handle already-tensors | ❌ Will re-wrap it               | ✅ Returns as-is if already tensor |

---

## ✅ TL;DR – When to use which?

| Use Case                              | Recommended |
|---------------------------------------|-------------|
| You want a fixed tensor (won’t change) | `tf.constant()` |
| Writing flexible, general code         | `tf.convert_to_tensor()` |

In [74]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A, shape=(4, 6))
C = tf.constant(numpy_A)
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(4, 6), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

--- 
### **Getting information from tensors**

#### When dealing with tensors we probably want to be aware of the following attributes:

| **Attribute** | **Meaning** | **Code** |
|---|---|---|
| **Shape** |  The length (number of elements) of each of the dimensions of a tensor. | `tensor.shape` |
| **Rank** | The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix is rank 2, a tensor has rank n. | `tensor.ndim` |
| **Axis or dimension** | A particular dimension of a tensor. | `tensor[0], tensor[:, 1]...` |
| **Size** | The total number of items in the tensor. | `tf.size(tensor)` |

In [76]:
# Create a rank 4 tensor (4 dimensions)\
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [77]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [84]:
print(
    rank_4_tensor.shape, '\n',
    rank_4_tensor.ndim, '\n',
    tf.size(rank_4_tensor))

(2, 3, 4, 5) 
 4 
 tf.Tensor(120, shape=(), dtype=int32)


In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  (3, 4, 5)


In [87]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)


In [90]:
# Get various attributes of our tensor
# Notice the final statement converted to NumPy

print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of a tensor: ", rank_4_tensor.shape)
# print("Elements along the 0 axis: ", rank_4_tensor[0].shape)  # wrong
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of a tensor:  (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


---
### Indexing tensors

#### Tensors can be indexed just like Python lists.

In [92]:
# aaron's reshaping the original matrix

rank_4_tensor = tf.reshape(tf.range(120), shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=int32, numpy=
array([[[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19]],

        [[ 20,  21,  22,  23,  24],
         [ 25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39]],

        [[ 40,  41,  42,  43,  44],
         [ 45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54],
         [ 55,  56,  57,  58,  59]]],


       [[[ 60,  61,  62,  63,  64],
         [ 65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74],
         [ 75,  76,  77,  78,  79]],

        [[ 80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94],
         [ 95,  96,  97,  98,  99]],

        [[100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114],
         [115, 116, 117, 118, 119]]]], dtype=int32)>

In [93]:
# Get the first 2 elements of each dimensions
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int32, numpy=
array([[[[ 0,  1],
         [ 5,  6]],

        [[20, 21],
         [25, 26]]],


       [[[60, 61],
         [65, 66]],

        [[80, 81],
         [85, 86]]]], dtype=int32)>

In [95]:
# Get the first element from each dimension from each index
#   excpet for the final one

print(rank_4_tensor[:1, :1, :1])

tf.Tensor([[[[0 1 2 3 4]]]], shape=(1, 1, 1, 5), dtype=int32)


In [96]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [97]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=int32, numpy=
array([[[[ 0],
         [ 5],
         [10],
         [15]]]], dtype=int32)>

In [98]:
rank_4_tensor[:1, :, :1, :1]

<tf.Tensor: shape=(1, 3, 1, 1), dtype=int32, numpy=
array([[[[ 0]],

        [[20]],

        [[40]]]], dtype=int32)>

In [99]:
rank_4_tensor[:, :1, :1, :1]

<tf.Tensor: shape=(2, 1, 1, 1), dtype=int32, numpy=
array([[[[ 0]]],


       [[[60]]]], dtype=int32)>

In [101]:
# Create a rank 2 tensor (2 dimensions)

rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
rank_2_tensor

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]], dtype=int32)>

In [102]:
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([4, 6]), 2)

In [ ]:
# Get the last item of each row of our rank 2 tensor

rank_2_tensor[-1, -1]  # wrong answer

<tf.Tensor: shape=(), dtype=int32, numpy=23>

In [ ]:
# Get the last item of each row of our rank 2 tensor

rank_2_tensor[-1:, -1:]  # wrong answer

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[23]], dtype=int32)>

In [ ]:
# Get the last item of each row of our rank 2 tensor
 
rank_2_tensor[:, -1]  

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5, 11, 17, 23], dtype=int32)>

In [107]:
rank_2_tensor(axis=1)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable

In [113]:
# Using `tf.gather` with `axis=1`, but wrong because tensor does not support -1 index
rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
print(rank_2_tensor)

last_items_of_each_row = tf.gather(rank_2_tensor, indices=[-1], axis=1)
print(last_items_of_each_row)
tf.squeeze(last_items_of_each_row)

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]], shape=(4, 6), dtype=int32)
tf.Tensor(
[[0]
 [0]
 [0]
 [0]], shape=(4, 1), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>

In [114]:
# You'll need to calculate the last index, but not using -1

# Using `tf.gather` with `axis=1`, but wrong because tensor does not support -1 index
rank_2_tensor = tf.reshape(tf.range(24), shape=(4, 6))
print(rank_2_tensor)

last_items_of_each_row = tf.gather(rank_2_tensor, indices=[6-1], axis=1)
print(last_items_of_each_row)
tf.squeeze(last_items_of_each_row)

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]], shape=(4, 6), dtype=int32)
tf.Tensor(
[[ 5]
 [11]
 [17]
 [23]], shape=(4, 1), dtype=int32)


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5, 11, 17, 23], dtype=int32)>

**TL;DR** for `tf.gather()` and `tf.squeeze()`:

---

## 🔹 `tf.gather()`

| Purpose                    | Grabs slices along an axis (like selecting rows or columns)        |
|----------------------------|---------------------------------------------------------------------|
| Common use                 | Select specific rows/columns (e.g., `axis=0` for rows, `axis=1` for columns) |
| Negative indices support?  | ❌ No — unlike NumPy, `-1` does **not** mean "last"                 |
| Fix for last index         | Use `tf.shape(tensor)[axis] - 1` instead of `-1`                   |
| Example                    | `tf.gather(tensor, indices=[2], axis=1)` → gets column 2 from all rows |

---

## 🔹 `tf.squeeze()`

| Purpose                    | Removes dimensions with size 1 (e.g., shape `[4, 1]` → `[4]`)        |
|----------------------------|-----------------------------------------------------------------------|
| When to use it             | After slicing/gathering when you want to flatten out unnecessary dims |
| Example                    | `tf.squeeze(tf.constant([[1], [2], [3]]))` → `[1, 2, 3]`             |
| Optional axis              | You can specify `axis` if you only want to squeeze specific dimensions |

---

### ✅ Example combo:

```python
last_col_index = tf.shape(tensor)[1] - 1
last_column = tf.gather(tensor, indices=[last_col_index], axis=1)
last_column = tf.squeeze(last_column)
```

---


In [115]:
# Add in extra dimension to our rank 2 tensor

rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(4, 6, 1), dtype=int32, numpy=
array([[[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5]],

       [[ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]],

       [[12],
        [13],
        [14],
        [15],
        [16],
        [17]],

       [[18],
        [19],
        [20],
        [21],
        [22],
        [23]]], dtype=int32)>

**TL;DR for `[..., tf.newaxis]`** 👇

---

## 🔹 `[..., tf.newaxis]`

| Purpose                          | Adds a **new dimension** (axis of size 1) at the end of the tensor |
|----------------------------------|---------------------------------------------------------------------|
| Equivalent to                    | `tf.expand_dims(tensor, axis=-1)`                                  |
| Useful for                       | - Adding channel dimensions (e.g., grayscale images)                |
|                                  | - Preparing shape for broadcasting or model input                   |
| Shape change                     | From `[a, b]` → `[a, b, 1]`                                         |
| Can also insert in other axes   | Yes! Use slicing like `tensor[:, tf.newaxis, :]` to insert at axis 1 |

---

## ✅ Example:

```python
x = tf.constant([1, 2, 3])        # shape: (3,)
x_expanded = x[..., tf.newaxis]   # shape: (3, 1)
```

---

### 🧠 Bonus: Why use `...`?

- `...` is a shortcut for "all previous dimensions"
- So `[..., tf.newaxis]` means: "add a new axis at the very end"

---


In [116]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand final axis


<tf.Tensor: shape=(4, 6, 1), dtype=int32, numpy=
array([[[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5]],

       [[ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11]],

       [[12],
        [13],
        [14],
        [15],
        [16],
        [17]],

       [[18],
        [19],
        [20],
        [21],
        [22],
        [23]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis=0)  # expand the 0-axis

<tf.Tensor: shape=(1, 4, 6), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]], dtype=int32)>

In [118]:
tf.expand_dims(rank_2_tensor, axis=1)  # expand the 1-axis

<tf.Tensor: shape=(4, 1, 6), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4,  5]],

       [[ 6,  7,  8,  9, 10, 11]],

       [[12, 13, 14, 15, 16, 17]],

       [[18, 19, 20, 21, 22, 23]]], dtype=int32)>

In [119]:
tf.expand_dims(rank_2_tensor, axis=0)
rank_2_tensor

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]], dtype=int32)>

---
### Manipulating tensors (tensor operations)

#### **Basic Operations**

`+, -, *, /`

In [120]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])

tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

#### Original tensor is unchanged!

In [122]:
# Original tensor is unchanged!!

tensor = tensor + 10
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [126]:
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [128]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [129]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [130]:
# Substraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

#### We can use the tensorflow built-in function too

In [131]:
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [132]:
# original tensor unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### **Matrix Multiplication**

#### - the aboved operation is `element-wise` operation, not like `matrix multiplication`
#### - In machine learning, matrix multiplication is one of the most common tensor operations - Like `dot product not necessarily element-wise`

> There are two rules our tensors (or matrices) need to fulfill if we're going to matrix multiply them:
> 1. The inner dimensions must match
> 2. The resulting matrix has the shape of the `outer` dimensions

In [134]:
# Matrix multiplication in tensorflow
print(tensor)
tf.linalg.matmul(tensor, tensor)
# tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [136]:
tensor * tensor  # this is element-wise

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

### Matrix multiplication with Python operator "@"

In [138]:
# matrix multiplication with Python operator "@"
#   it's called matrix multiplication
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [139]:
tensor.shape

TensorShape([2, 2])

In [145]:
# Create a tensor (3, 2) shape
X = tf.reshape(tf.range(1, 7), shape=(3, 2))
X

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [146]:
# Create another (3, 2) tensor
Y = tf.reshape(tf.range(7, 13), shape=(3, 2))
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [147]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [148]:
# Try to matrix multiply tensors of same shape
X @ Y

2025-03-24 20:13:34.242127: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2]


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [149]:
tf.matmul(X, Y)

2025-03-24 20:13:56.175880: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2]


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [151]:
# to make the two matrices which being multiplied "match"
X_T = tf.transpose(X)
X_T

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 3, 5],
       [2, 4, 6]], dtype=int32)>

In [152]:
X_T @ X

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[35, 44],
       [44, 56]], dtype=int32)>

In [153]:
X @ X_T

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 5, 11, 17],
       [11, 25, 39],
       [17, 39, 61]], dtype=int32)>

### Dot Product
![Dot Product](./image/2025-03-24-20-30-20.png)

---
##### reference vid sec 1 / 21 04:35

In [155]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [156]:
tf.transpose(Y)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  9, 11],
       [ 8, 10, 12]], dtype=int32)>

In [159]:
# observe the shapes
X.shape, tf.reshape(Y, shape=(2, 3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [160]:
# Try to multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [161]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [163]:
# try to change shape of X, instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [164]:
tf.reshape(X, shape=(2, 3)).shape, Y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

### Transpose works for matmul, BUT NOTICE, contents are different

In [ ]:
# Can do the same with transpose, but contents different
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [166]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Can do the same with transpose (however, result differs)
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [168]:
# Try matrix multiplication with transpose rather than reshape
X.shape, Y.shape

(TensorShape([3, 2]), TensorShape([3, 2]))

In [169]:
tf.transpose(X) @ Y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [170]:
X @ tf.transpose(Y)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [171]:
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [172]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

---
### 📓 **Resources** [Info and example of matrix multiplication](https://www.mathisfun.com/algebra/matrix-multiplying.html)

---

### **The dot product**

We can perform matrix multiplication using:
```
* tf.matmul()
* tf.tensordot()

In [175]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [176]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
# (require X or Y to be transpose

tf.tensordot(tf.transpose(X), Y)

TypeError: Missing required positional argument

In [180]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [181]:
tf.tensordot(tf.transpose(X), Y, axes=0)

<tf.Tensor: shape=(2, 3, 3, 2), dtype=int32, numpy=
array([[[[ 7,  8],
         [ 9, 10],
         [11, 12]],

        [[21, 24],
         [27, 30],
         [33, 36]],

        [[35, 40],
         [45, 50],
         [55, 60]]],


       [[[14, 16],
         [18, 20],
         [22, 24]],

        [[28, 32],
         [36, 40],
         [44, 48]],

        [[42, 48],
         [54, 60],
         [66, 72]]]], dtype=int32)>

In [182]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [183]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [184]:
# Check the values of Y, reshape Y and transposed Y

print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


---
#### Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose rather than reshape one of the tensors to get satisfy the matrix multiplication

---
---
### Changing the datatype of the tensor

In [185]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [186]:
C = tf.constant([7, 10])
C.dtype

tf.int32

### float32, float16, bfloat16

In [189]:
D = tf.cast(B, dtype=tf.float16)
D

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>

In [190]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [191]:
E_float = tf.cast(E, dtype=tf.float16)
E_float

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

---
### Aggregating Tensors


#### Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

### 🦜 Bilingual Notes: Agregating Tensors

---
### 🔹 What does **"aggregating"** mean in TensorFlow?

In TensorFlow, **"aggregating"** refers to **combining or summarizing multiple values**—for example, summing them, averaging them, or performing other reduction operations. This is used extensively in:

- Loss calculation
- Metric computation
- Gradient updates
- Distributed training

---

### 🔸 Simple Example: `tf.reduce_mean`

```python
import tensorflow as tf

x = tf.constant([1.0, 2.0, 3.0, 4.0])
mean = tf.reduce_mean(x)
print(mean)  # Output: 2.5
```

Here, `reduce_mean` **aggregates** all elements in the tensor and returns their average. Similar functions include `tf.reduce_sum`, `tf.reduce_max`, etc.

---

### 🔸 Practical Example: Aggregating metrics with `tf.keras.metrics.Mean`

```python
m = tf.keras.metrics.Mean()

m.update_state([1, 2, 3])
m.update_state([4, 5])

print("Mean result:", m.result().numpy())  # Output: 3.0
```

Here, `Mean()` is an aggregator. Every time you call `update_state()`, it collects more values, and `result()` computes the overall average.

---

### 🔸 Aggregation in Distributed Training

If you're using multiple GPUs (or TPUs), TensorFlow will **aggregate values (e.g., gradients, loss) across devices** so that training remains consistent.

Example:

```python
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    def compute_loss(labels, predictions):
        per_example_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=64)
```

`compute_average_loss` aggregates per-example losses across devices to get a correct average loss for training.

---

### ✅ Common Use Cases of Aggregating in TensorFlow

| Use Case | Description |
|----------|-------------|
| **Loss Calculation** | Combine per-sample losses into a single scalar |
| **Metric Computation** | Accumulate metrics (e.g., accuracy) across batches |
| **Gradient Updates** | Combine gradients across GPUs in distributed training |
| **Model Predictions** | Aggregate predictions from multiple models (e.g., ensemble averaging) |





---
#### 在 TensorFlow 裡面，「**aggregating**」這個詞的意思是 **彙總、聚合**，也就是將多個值（可能來自不同來源、批次、張量等）進行合併、統整成一個值或一個結構的動作。這個操作在許多場景都很重要，特別是在 **分散式訓練（Distributed Training）**、**loss 計算**、**梯度更新** 和 **指標評估（metrics evaluation）** 中。

---

### 🔹 一個簡單的例子：`tf.reduce_mean`

```python
import tensorflow as tf

x = tf.constant([1.0, 2.0, 3.0, 4.0])
mean = tf.reduce_mean(x)
print(mean)  # Output: 2.5
```

這裡 `reduce_mean` 就是對張量 `x` 執行聚合操作：把所有元素「彙總」起來取平均。類似的還有 `tf.reduce_sum`、`tf.reduce_max` 等。

---

### 🔹 更實用的例子：在 `tf.keras.metrics.Mean` 中使用 aggregation

```python
m = tf.keras.metrics.Mean()

m.update_state([1, 2, 3])
m.update_state([4, 5])

print("Mean result:", m.result().numpy())  # Output: 3.0
```

這裡 `Mean()` 指標會 **聚合（aggregate）多次輸入的值**，在訓練過程中你可以多次呼叫 `update_state()` 累積資料，最後呼叫 `result()` 得到整體的平均。

---

### 🔹 在分散式訓練中常見的 aggregation：`tf.distribute.Strategy`

假設你有多個 GPU，在每個 GPU 上訓練得到的 loss 或梯度可能不同，TensorFlow 會自動 **聚合（aggregate）所有設備的 loss/gradient**，確保你更新的權重是基於整體資訊。

範例如下：

```python
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    def compute_loss(labels, predictions):
        per_example_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=64)
```

這裡 `compute_average_loss` 就是一種 aggregation，把所有設備上計算出的 loss 進行平均。

---

### ✅ Aggregating 的常見用途：

| 場景 | 說明 |
|------|------|
| **loss 計算** | 多個樣本的 loss 聚合成一個平均 loss |
| **metrics 評估** | 多個 batch 的 accuracy、precision 統計彙總 |
| **梯度更新** | 分散式環境中各個 device 計算出的梯度進行加總或平均 |
| **多輸入彙總** | 合併不同資料來源的值，例如多模型的預測結果進行加權平均 |

---

#### Aggregating Examples:

In [193]:
# Get the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [194]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

### Let's go through the following forms of aggregation
- Get the minimum
- Get the maximum
- Get the mean of tensor
- Get the sum of tensor

In [195]:
# Create a random tensor with values betwen 0 and 100 of size 100
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([99,  5, 60, 99, 50, 38, 40, 20, 21, 18, 42, 84, 82, 16, 65, 59, 19,
       35, 62,  1, 53, 43, 87, 29, 45, 48, 74,  1, 42, 50, 60, 12, 16, 15,
       44, 42, 64, 66, 52, 86, 22, 25, 11,  1, 87,  4, 28, 21, 27, 57])>

In [196]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [197]:

# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [198]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=99>

In [199]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [200]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2127>

---
### 🛠️ **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods
---

In [ ]:
# 计算方差
variance_E = tf.math.reduce_variance(E)
print("Variance of E:", variance_E.numpy())

# 计算标准差
std_dev_E = tf.math.reduce_std(E)
print("Standard Deviation of E:", std_dev_E.numpy())

In [201]:
E = tf.random.randint(0, 100, size=50)
print(f"E Variance: {tf.reduce_variance(E)}")
print(f"E variance (in NumPy): {tf.reduce_variance(E).numpy()}")

print(f"E Standard Deviation: {tf.math.reduce_std(E)}")
print(f"E Standard Deviation (in NumPy): {tf.math.reduce_std(E).numpy()}")

AttributeError: module 'tensorflow._api.v2.random' has no attribute 'randint'

In [202]:
E = tf.constant(np.random.randint(0, 100, size=50))
print(f"E Variance: {tf.reduce_variance(E)}")
print(f"E variance (in NumPy): {tf.reduce_variance(E).numpy()}")

print(f"E Standard Deviation: {tf.math.reduce_std(E)}")
print(f"E Standard Deviation (in NumPy): {tf.math.reduce_std(E).numpy()}")

AttributeError: module 'tensorflow' has no attribute 'reduce_variance'

In [204]:
E = tf.constant(np.random.randint(0, 100, size=50))
print(f"E Variance: {tf.math.reduce_variance(E)}")
print(f"E variance (in NumPy): {tf.math.reduce_variance(E).numpy()}")

print(f"E Standard Deviation: {tf.math.reduce_std(E)}")
print(f"E Standard Deviation (in NumPy): {tf.math.reduce_std(E).numpy()}")

TypeError: Input must be either real or complex. Received integer type <dtype: 'int64'>.

### ✅ Fix: Cast our tensor to `tf.float32` or `tf.float64`

In [210]:
import tensorflow as tf
import numpy as np

E = tf.constant(np.random.randint(0, 100, size=50), dtype=tf.float32)
print(E)

print(f"E Variance: {tf.math.reduce_variance(E)}")
print(f"E variance (in NumPy): {tf.math.reduce_variance(E).numpy()}")

print(f"E Standard Deviation: {tf.math.reduce_std(E)}")
print(f"E Standard Deviation (in NumPy): {tf.math.reduce_std(E).numpy()}")

tf.Tensor(
[26. 29. 98. 21. 91. 36. 81. 34. 65. 40. 92.  2. 79. 56. 29. 23. 95. 15.
 47.  1. 15. 16. 95.  3. 51.  0. 54. 15. 62.  4. 77. 64. 94. 86. 94. 58.
 99. 64. 78.  0. 43. 68. 35. 18. 67. 72. 65. 81. 72. 34.], shape=(50,), dtype=float32)
E Variance: 965.8656005859375
E variance (in NumPy): 965.8656005859375
E Standard Deviation: 31.078378677368164
E Standard Deviation (in NumPy): 31.078378677368164


In [208]:
tf.math.reduce_variance(E)

<tf.Tensor: shape=(), dtype=float32, numpy=743.3504>

In [209]:
tf.math.reduce_variance(E).numpy()

743.3504

---
### Aaron's comments on `Variance` and `Standard Deviation`:
> Our results has something like below:
> ```text
> E Variance: 965.86
> E Standard Deviation: 31.08
> ```

### 🤔 But our numbers of `E` are between 0 and 100?

Yes, and that **doesn't contradict** the result — below is why.

---

### 🧠 Variance is not bounded by the data range

The **variance** measures how spread out the values are from the **mean**.

The formula is:

\[
\text{Variance} = \frac{1}{N} \sum_{i=1}^{N} (x_i - \bar{x})^2
\]

Even if all numbers are in `[0, 100)`, if they're very **spread out**, then the squared differences from the mean will be large.

---

### 📊 Example

Say our values are roughly spread from 0 to 100. The **mean** will be around 50. The squared difference from 0 to 50 is:

\[
(0 - 50)^2 = 2500
\]

Even a small number of such values will heavily affect the variance. But the **variance is the *mean* of those squared differences**, so even if the squared errors go from, say, 0 to 2500, the average can be around **900–1000** easily.

---

### ✅ Key insight:

If our data is **uniformly random between 0 and 100**, the theoretical population **variance** is:

\[
\text{Var}(U(0, 100)) = \frac{(b - a)^2}{12} = \frac{(100 - 0)^2}{12} = \frac{10000}{12} \approx 833.33
\]

Our result:

```
965.86
```

is **very close to this**, considering sampling randomness from just 50 numbers!

---

### ✅ And for Standard Deviation:

\[
\text{Std} = \sqrt{Variance} \approx \sqrt{965} \approx 31
\]

Which matches our output: `31.078`

---

### 🧪 Quick check in NumPy


In [212]:
data = np.random.randint(0, 100, size=50)
print(np.var(data), "\n", np.std(data))

692.6035999999999 
 26.31736308979302


Will likely give something close to:

```text
~900-1000 variance, ~30-32 std
```

---

In [213]:
# Find the variance of our tensor
tf.reduce_var(E)

AttributeError: module 'tensorflow' has no attribute 'reduce_var'

In [215]:
# Find the variance of our tensor, with the access to tensorflow_probability

import tensorflow_probability as tfp
tfp.stats.variance(E)

ModuleNotFoundError: No module named 'tensorflow_probability'

In [219]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=float16))

NameError: name 'float16' is not defined

In [217]:
tf.math.reduce_variance(E)

<tf.Tensor: shape=(), dtype=float32, numpy=965.8656>

In [220]:
tf.math.reduce_std(tf.cast(E, dtype=tf.float16))

<tf.Tensor: shape=(), dtype=float16, numpy=31.08>

In [221]:
tf.math.reduce_variance(tf.cast(E, dtype=tf.bfloat16))

<tf.Tensor: shape=(), dtype=bfloat16, numpy=964>

---
---
### Find the positional maximum and minimum

### 🔹 Aaron's comments on **Positional Maximum / Minimum**

In TensorFlow (and NumPy), **positional maximum/minimum** refers to **finding the index (position)** where the **maximum or minimum value** occurs in a tensor or array — not the value itself, but *where* it is.

| Function        | Returns               | Meaning                        |
|----------------|------------------------|--------------------------------|
| `tf.argmax()`  | Index of max value     | Positional maximum             |
| `tf.argmin()`  | Index of min value     | Positional minimum             |
| `tf.reduce_max()` | Actual max value    | Not positional                 |
| `tf.reduce_min()` | Actual min value    | Not positional                 |

---

### 🧠 Examples:

In [227]:
# 1D tensor
import tensorflow as tf
x = tf.constant([3, 7, 2, 9, 5])

In [230]:
# ✅ Positional Maximum → `tf.argmax()`
print(f"x: {x}")
max_pos = tf.argmax(x)
print(f"max_pos: {max_pos.numpy()}") # Output: 3 → because 9 is the max, and it's at index 3

x: [3 7 2 9 5]
max_pos: 3


In [ ]:

# ✅ Positional Minimum → `tf.argmin()`
print(f"x: {x}")

min_pos = tf.argmin(x)
print(f"main_pos: {min_pos.numpy()}") # Output: 2 → because 2 is the min, and it's at index 2

x: [3 7 2 9 5]
main_pos: 2


In [236]:

# 🧩 Multidimensional Tensor Example

x2d = tf.constant([[1, 9, 3],
                   [7, 2, 8]])
print(f"x2d:\n {x2d}\n")

print(f"tf.argmax(x2d): {tf.argmax(x2d, axis=0)}, because → argmax across rows for each column\n")  # → argmax across rows for each column
print(f"tf.argmax(x2d): {tf.argmax(x2d, axis=1)}, because → argmax across rows for each row\n")  # → argmax across rows for each row


x2d:
 [[1 9 3]
 [7 2 8]]

tf.argmax(x2d): [1 0 1], because → argmax across rows for each column

tf.argmax(x2d): [1 2], because → argmax across rows for each row



#### End of Aaron's Comments on Position Max/Min
---

In [239]:
# Create a new tensor for finding positional min and max

tf.random.set_seed(42)

F = tf.random.uniform(shape=[10])
F

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686],
      dtype=float32)>

In [240]:
# Find the pos max
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=7>

In [242]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.8724445>

In [241]:
# Find the pos min
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=4>

In [243]:
# Index on our smallest value position
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.03366041>

In [249]:
F[tf.argmax(F)].numpy(), F[tf.argmin(F)].numpy()

(0.8724445, 0.03366041)

In [250]:
# check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [251]:
# Check for Equality
F[tf.argmin(F)] == tf.reduce_min(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

---
---
### Squeezing a Tensor （Removing all single dimensions)

In [252]:
# Create a tensor to get started
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))

In [253]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [257]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.68789124, 0.48447883, 0.9309944 , 0.252187  , 0.73115396,
        0.89256823, 0.94674826, 0.7493341 , 0.34925628, 0.54718256,
        0.26160395, 0.69734323, 0.11962581, 0.53484344, 0.7148968 ,
        0.87501776, 0.33967495, 0.17377627, 0.4418521 , 0.9008261 ,
        0.13803864, 0.12217975, 0.5754491 , 0.9417181 , 0.9186585 ,
        0.59708476, 0.6109482 , 0.82086265, 0.83269787, 0.8915849 ,
        0.01377225, 0.49807465, 0.57503664, 0.6856195 , 0.75972784,
        0.908944  , 0.40900218, 0.8765154 , 0.53890026, 0.42733097,
        0.401173  , 0.66623247, 0.16348064, 0.18220246, 0.97040176,
        0.06139731, 0.53034747, 0.9869994 , 0.4746945 , 0.8646754 ],
       dtype=float32)>,
 TensorShape([50]))

---
---
### One-Hot Encoding

In [259]:
# Create a list of indices
some_list = [0, 1, 2, 3]  # could be red, green, blue, purple

# One hot encode out list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [260]:
# specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="yo I love deep learning", off_value="so sorry" )

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'so sorry', b'so sorry',
        b'so sorry'],
       [b'so sorry', b'yo I love deep learning', b'so sorry',
        b'so sorry'],
       [b'so sorry', b'so sorry', b'yo I love deep learning',
        b'so sorry'],
       [b'so sorry', b'so sorry', b'so sorry',
        b'yo I love deep learning']], dtype=object)>

---
### Squaring, log, square root

In [261]:
# Create a new tensor 
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [262]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [263]:
# Find the squaretoot
tf.sqrt(H)

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt] name: 

In [264]:
# Find the squareroot - change to float
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [265]:
# Find the lgo
tf.math.log(H)

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Log}}; Op<name=Log; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Log] name: 

In [266]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([0.    , 0.6934, 1.099 , 1.387 , 1.609 , 1.792 , 1.946 , 2.08  ,
       2.197 ], dtype=float16)>

---
---
### Tensors and Numpy

#### TensorFlow interacts beautifully with Numpy arrays

#### 🔑 **Note:** One of the main deffierences between a TensorFlow tensor and a NumPy array is that a TensoprFlow tensor can be run on a GPU or TPU (for faster numerical processing)

In [267]:
# Create a tensor directly from a numpy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [268]:
# Convert out tensor back to a numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [269]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [270]:
# The default tyupes of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check dtype of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding Access to GPU

### Access to GPU for `Mac` `M1`, `M2`, `M3` Apple Silicon GPU

In [ ]:
#check for gpu for Pytorch
if torch.backends.mps.is_available():
   mps_device = torch.device("mps")
   x = torch.ones(1, device=mps_device)
   print (x)
else:
   print ("MPS device not found.")

NameError: name 'torch' is not defined

In [281]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import tensorflow as tf
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [282]:
import tensorflow as tf
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

print('-----------------------')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print(tf.config.experimental.get_memory_info('GPU:0'))
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
-----------------------
{'current': 0, 'peak': 0}
GPU details:  {'device_name': 'METAL'}


In [279]:
if tf.config.list_physical_devices('GPU'):
  # Returns a dict in the form {'current': <current mem usage>,
  #                             'peak': <peak mem usage>}
  print(tf.config.experimental.get_memory_info('GPU:0'))

{'current': 0, 'peak': 0}


In [288]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [289]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [290]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [291]:
!open -a "Activity Monitor"

In [292]:
!top

>    WindowManage 5.3  22:27.42 5      2    339   37M- 5M-   57273  K[19;23H99+77.0/1564 23/1618.85584K-21H459-+total, 640K6501689345(60) swapins, 11321832(0) swapouts125838667T MEM    PURG   CMPRS  PGRP  30283.297.29.420 total, 0B489320413(68698942772941 stuck, 578 sleeping, 5640 threads946, 4.65, 4.04369.327.30514620733(209670848753.58805+ 1896M-032.58/1020.195+2581 sleeping, 5633 threads         205.5511.98% sys, 72.46% idle451595588069(636) swapins, 11321832(0) swapout232346108271 stuck, 575 sleeping, 5625 threads29023851762669161(92) swapins, 11321832(0) swapouts48776892958577 sleeping, 5694 threads         39.693.5566.7591 total, 2704K resident, 69M private, 1325M shared.28127760317(156) swapins, 11322392(560) swapo31980434815001543829474, 4.74, 4.072.579.14% sys, 78.280 total, 0B[0400433(16) swapins, 11323240(848) swapou857197439714 running, 573 sleeping, 5699 threads56.9512.80.9666 total, 2128K resident, 69M private, 1326M shared.4851165(320) swapouts467944562570903770564.989

In [297]:
# this is a typo - Display has s => Displays => SPDisplay`s`DataType
!system_profiler SPDisplayDataType


In [294]:
!system_profiler SPDisplaysDataType


Graphics/Displays:

    Apple M1 Pro:

      Chipset Model: Apple M1 Pro
      Type: GPU
      Bus: Built-In
      Total Number of Cores: 16
      Vendor: Apple (0x106b)
      Metal Support: Metal 3
      Displays:
        Color LCD:
          Display Type: Built-in Liquid Retina XDR Display
          Resolution: 3024 x 1964 Retina
          Main Display: Yes
          Mirror: Off
          Online: Yes
          Automatically Adjust Brightness: Yes
          Connection Type: Internal
        VA2403-FHD:
          Resolution: 1920 x 1080 (1080p FHD - Full High Definition)
          UI Looks like: 1920 x 1080 @ 60.00Hz
          Mirror: Off
          Online: Yes
          Rotation: Supported



In [298]:
!system_profiler SPDisplayDataType

---
### ** Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible